In [2]:
import pandas as pd
import numpy as np
#testserdar

In [7]:
wsp_contract = pd.read_excel("../../Datasets/study-1-data.xlsx")



In [8]:
wsp_contract['Sales'] = wsp_contract[['Demand', 'Stock']].min(axis=1)

In [9]:
wsp_contract['Leftovers'] = wsp_contract['Stock'] - wsp_contract['Sales']

In [10]:
wsp_contract['Unmet_Demand'] = wsp_contract['Demand'] - wsp_contract['Sales']

In [11]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Unnamed: 5,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,NaN,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,NaN,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,NaN,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,NaN,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,NaN,Retailer 1,60,60,0,75


In [12]:
wsp_contract.drop(columns=["Unnamed: 5"], inplace=True)

In [13]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75


In [14]:
wsp_contract['Realized_Mfg_Profit'] = (wsp_contract['Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Stock']) 

In [15]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300


In [16]:
wsp_contract['Total_Revenue'] = wsp_contract['Stock'] * wsp_contract['Wholesale p.']

In [17]:
wsp_contract['Realized_Mfg_Profit_Share'] = wsp_contract.apply(lambda row: row['Realized_Mfg_Profit'] / row['Total_Revenue'] if row['Total_Revenue'] != 0 else 0, axis=1)

In [18]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000


In [19]:

wsp_contract['Optimal_Stock'] = 100*((12-wsp_contract['Wholesale p.'])/12) + 50
wsp_contract['Optimal_Stock'] = wsp_contract['Optimal_Stock'].astype(int)

In [20]:
wsp_contract['Under/Overstock_wrt_Optimal'] = wsp_contract['Stock'] - wsp_contract['Optimal_Stock']

In [21]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,83,-23
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,91,-11
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,91,-11
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,75,-24
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,83,-23


In [22]:
potential_demand = np.arange(50, 151)
expected_sales = []

for index, row in wsp_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Stock'])

        cumulative_expected_sales += sales
        
        if sales == row['Stock']:
            cumulative_expected_sales += (150 - demand_realization) * row['Stock']
            break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

wsp_contract['Expected Sales'] = np.array(expected_sales)

In [23]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,83,-23,59.455446
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,91,-11,75.396040
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,91,-11,75.396040
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,75,-24,50.990099
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,83,-23,59.455446
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,300,480,0.625000,83,-23,59.455446
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,0,0,0.000000,58,-58,0.000000
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,306,459,0.666667,75,-24,50.990099
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,0,0,0.000000,50,-50,0.000000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,300,480,0.625000,83,-23,59.455446


In [24]:
wsp_contract["Expected_Leftovers"] = wsp_contract["Optimal_Stock"] - wsp_contract["Expected Sales"]

In [25]:
wsp_contract['Expected Retailer Profit'] = wsp_contract['Expected Sales'] * wsp_contract['Wholesale p.'] - (3 * wsp_contract['Expected Sales']) #nasıl yapacaz kaça sattığını bilmiyoz

In [26]:
wsp_contract['Expected Mfg Profit'] = (wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Optimal_Stock'])

In [27]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,83,-23,59.455446,23.544554,297.277228,415
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,91,-11,75.396040,15.603960,301.584158,364
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,91,-11,75.396040,15.603960,301.584158,364
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,75,-24,50.990099,24.009901,305.940594,450
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,83,-23,59.455446,23.544554,297.277228,415


In [28]:
wsp_contract['Total Expected Revenue'] = wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']

wsp_contract['Expected Mfg. Profit Share'] = wsp_contract.apply(lambda row: row['Expected Mfg Profit'] / row['Total Expected Revenue'] if row['Total Expected Revenue'] != 0 else 0, axis=1)

In [29]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Total Expected Revenue,Expected Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,560,0.571429,91,-11,75.396040,15.603960,301.584158,364,637,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,560,0.571429,91,-11,75.396040,15.603960,301.584158,364,637,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,459,0.666667,75,-24,50.990099,24.009901,305.940594,450,675,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,...,0,0.000000,58,-58,0.000000,58.000000,0.000000,464,638,0.727273
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,...,459,0.666667,75,-24,50.990099,24.009901,305.940594,450,675,0.666667
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,...,0,0.000000,50,-50,0.000000,50.000000,0.000000,450,600,0.750000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000


In [30]:
#wsp_contract.to_csv("../../Datasets/study-1-data-wsp.csv", index=False)
wsp_contract['Predicted Sales'] = wsp_contract[['Demand', 'Optimal_Stock']].min(axis=1).astype(int)

In [31]:
wsp_contract['Predicted Leftovers'] = wsp_contract['Optimal_Stock'] - wsp_contract['Predicted Sales']

In [32]:
wsp_contract['Predicted Retailer Profit'] = wsp_contract['Predicted Sales'] * wsp_contract['Wholesale p.'] - (3 * wsp_contract['Predicted Sales'])
wsp_contract['Predicted Mfg Profit'] = (wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Optimal_Stock'])

In [33]:
wsp_contract['Realized Retailer Profit'] = wsp_contract['Sales'] * 12 - (wsp_contract['Stock'] * wsp_contract['Wholesale p.'])

In [34]:
wsp_contract['Predicted Mfg. Profit Share'] = wsp_contract.apply(
    lambda row: row['Predicted Mfg Profit'] / (row['Optimal_Stock'] * row['Wholesale p.']) if (row['Optimal_Stock'] * row['Wholesale p.']) != 0 else 0,
    axis=1
)

In [31]:
#wsp_contract.to_csv("../../Datasets/study-1-data-wsp.csv", index=False)

In [37]:
grouped_data_retailer = wsp_contract.groupby('Player ID')


In [43]:
for group_key, group_data in grouped_data_retailer:
    group_data['Retailer Profit Mean Of All 40 Periods'] = group_data['Realized Retailer Profit'].mean()
    group_data['Manufacturer Profit Mean Of All 40 Periods'] = group_data['Realized_Mfg_Profit'].mean()


In [44]:
wsp_contract.iloc[360]

Experiment                                   3
Period                                       1
Player ID                      Manufacturer 10
Demand                                     122
Wholesale p.                                 9
Player ID.1                        Retailer 10
Stock                                       90
Sales                                       90
Leftovers                                    0
Unmet_Demand                                32
Realized_Mfg_Profit                        540
Total_Revenue                              810
Realized_Mfg_Profit_Share             0.666667
Optimal_Stock                               75
Under/Overstock_wrt_Optimal                 15
Expected Sales                       81.881188
Expected_Leftovers                   -6.881188
Expected Retailer Profit            491.287129
Expected Mfg Profit                        450
Total Expected Revenue                     675
Expected Mfg. Profit Share            0.666667
Predicted Sal

In [54]:
#Null Hypothesis (H0): The mean order quantity made by retailers is not significantly different from the optimal stock level.

#Alternative Hypothesis (H1): The mean order quantity made by retailers is significantly different from the optimal stock level.

#Reasoning: Retailers are expected to be able to adapt correctly to the changing wholesale price as they are rational agents.

wsp_contract['null'] = 0
from scipy.stats import wilcoxon
_, p_value = wilcoxon(abs(wsp_contract["Under/Overstock_wrt_Optimal"]), wsp_contract['null'])
print("p value:",p_value)
if p_value < 0.05:
    print("Reject the null hypothesis: The median absolute difference is significantly different from zero.")
else:
    print("Fail to reject the null hypothesis: No significant difference from zero in the median absolute difference.")

p value: 6.811157769659205e-143
Reject the null hypothesis: The median absolute difference is significantly different from zero.


In [128]:
my = []
i = 0
for group_key, group_data in grouped_data_retailer:
    a=group_data.iloc[0:5]['Expected Retailer Profit'].mean()
    b=group_data.iloc[20:30]['Expected Retailer Profit'].mean()
    my.append([a, b])
    i+=1


In [129]:
my = pd.DataFrame(my)

In [130]:
my

,0,1
0,300.732673,299.475248
1,475.108911,450.246535
2,288.089109,412.252475
3,254.637624,333.886139
4,324.168317,335.623762
5,233.287129,266.435644
6,361.770297,301.940594
7,365.316832,375.420792
8,204.277228,368.490099
9,125.000000,286.806931


In [152]:
#hypo 2
#H0: No learning


t_stat, p_value = test_rel(my[0], my[1])
print("P value is", p_value, "which is higher than 0.05.\nMeaning that the null hypothesis is accepted.")

P value is 0.17795817163976257 which is higher than 0.05.
Meaning that the null hypothesis is accepted.


In [135]:
#hypo 3: 

#null: no difference between the best expected retailer profit scorers in under/over ordering stat
#reject null in the end


liste = []
i = 0
for group_key, group_data in grouped_data_retailer:
    a=group_data['Expected Retailer Profit'].mean()
    liste.append([a, i])
    i+=1


In [136]:
liste

[[261.0556930693069, 0],
 [439.4787128712871, 1],
 [375.1101485148515, 2],
 [332.6965346534654, 3],
 [335.029702970297, 4],
 [218.45915841584156, 5],
 [323.5022277227723, 6],
 [370.56336633663364, 7],
 [369.53341584158414, 8],
 [247.56188118811883, 9],
 [312.84282178217825, 10],
 [362.07400990099006, 11],
 [342.4772277227723, 12],
 [374.62301980198015, 13],
 [192.64455445544553, 14],
 [291.67549504950495, 15],
 [384.47351485148516, 16],
 [274.5378712871287, 17],
 [379.68168316831685, 18],
 [278.36881188118815, 19],
 [330.6289603960396, 20],
 [483.2425742574258, 21]]

In [139]:
liste = sorted(liste, key=lambda x: x[0], reverse=True)

In [140]:
liste

[[483.2425742574258, 21],
 [439.4787128712871, 1],
 [384.47351485148516, 16],
 [379.68168316831685, 18],
 [375.1101485148515, 2],
 [374.62301980198015, 13],
 [370.56336633663364, 7],
 [369.53341584158414, 8],
 [362.07400990099006, 11],
 [342.4772277227723, 12],
 [335.029702970297, 4],
 [332.6965346534654, 3],
 [330.6289603960396, 20],
 [323.5022277227723, 6],
 [312.84282178217825, 10],
 [291.67549504950495, 15],
 [278.36881188118815, 19],
 [274.5378712871287, 17],
 [261.0556930693069, 0],
 [247.56188118811883, 9],
 [218.45915841584156, 5],
 [192.64455445544553, 14]]

In [143]:
first_indexes = [21, 1, 16, 18, 2]
last_indexes = [17, 0, 9, 5, 14]
df_best = pd.DataFrame()
df_worst = pd.DataFrame()

i = 0
for group_key, group_data in grouped_data_retailer:
    if i in first_indexes:
        df_best = pd.concat([df_best, group_data])
    elif i in last_indexes:
        df_worst = pd.concat([df_worst, group_data])
    i+=1

In [146]:
from scipy.stats import wilcoxon

In [149]:
res = wilcoxon(abs(df_best['Under/Overstock_wrt_Optimal']) ,abs(df_worst['Under/Overstock_wrt_Optimal']))  

In [150]:
res

WilcoxonResult(statistic=7102.0, pvalue=0.0018774414138789553)